<pre>
There are two ways to implement subgraphs in langgraphs:
1. Create parent and child langgraphs separately, then link them.
2. Define subgraphs directly within a single langgraph using nested structures.
</pre>
### <code>Method 1: Separate Langgraphs</code>


In [ ]:
from typing import TypedDict,Annotated
from dotenv import load_dotenv
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

In [ ]:
subgraph_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

class SubGraphState(TypedDict):
  text: str
  translated_text: str
  
def subgraph_translate_node(state:SubGraphState):
  response = subgraph_model.invoke(
    f"Translate the following text to Gujarati:\n\n{state['text']}"
  )
  return {"translated_text": response.content}

subgraph_builder = StateGraph(SubGraphState)
subgraph_builder.add_node("translate",subgraph_translate_node)
subgraph_builder.add_edge(START,"translate")
subgraph_builder.add_edge("translate",END)

subgraph = subgraph_builder.compile()

In [ ]:
parent_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

class ParentGraphState(TypedDict):
  text: str
  genrated_text: str
  translated_text: str
  
def parent_generate_node(state:ParentGraphState):
  response = parent_model.invoke(
    f"Generate a short story about the following topic:\n\n{state['text']}"
  )
  return {"genrated_text": response.content}

def parent_translate_node(state:ParentGraphState):
  response = subgraph.invoke({
    "text": state["text"]
  })
  return {"translated_text": response["translated_text"]}


parent_builder = StateGraph(ParentGraphState)
parent_builder.add_node("generate",parent_generate_node)
parent_builder.add_node("translate",parent_translate_node)
parent_builder.add_edge(START,"generate")
parent_builder.add_edge("generate","translate")
parent_builder.add_edge("translate",END)

parent_graph = parent_builder.compile()
result = parent_graph.invoke({
  "text": "DOGS"
})
result

### <code>Method 2: Nested Subgraphs</code>

In [25]:
class ParentGraphState(TypedDict):
  text: str
  answer_eng: str
  answer_guj: str

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [26]:
def subgraph_translate_node(state:ParentGraphState):
  response = model.invoke(
    f"Translate the following text to Gujarati without any other language:\n\n{state['answer_eng']}"
  )
  return {"answer_guj": response.content}

subgraph_builder = StateGraph(ParentGraphState)
subgraph_builder.add_node("translate",subgraph_translate_node)
subgraph_builder.add_edge(START,"translate")
subgraph_builder.add_edge("translate",END)
subgraph = subgraph_builder.compile()

In [27]:
def parent_generate_node(state:ParentGraphState):
  response = model.invoke(
    f"Generate a short story about the following topic:\n\n{state['text']}"
  )
  return {"answer_eng": response.content}

parent_builder = StateGraph(ParentGraphState)
parent_builder.add_node("generate",parent_generate_node)
parent_builder.add_node("translate",subgraph)  # Using subgraph as a node instead of a function
parent_builder.add_edge(START,"generate")
parent_builder.add_edge("generate","translate")
parent_builder.add_edge("translate",END)

parent_graph = parent_builder.compile()
result = parent_graph.invoke({
  "text": "DOGS"
})
result

{'text': 'DOGS',
 'answer_eng': 'Eleanor\'s days were a predictable rhythm of tea, crosswords, and tending her small rose garden. Her house, neat and quiet, echoed only with the gentle tick-tock of the grandfather clock in the hall. It had been this way for years, a comfortable solitude she rarely questioned.\n\nThen came Buster.\n\nHer niece, Sarah, had practically dragged her to the local shelter, insisting Eleanor needed "some company." Eleanor had resisted, citing her age, her routine, her perfectly adequate peace. But then she saw him: a scruffy terrier mix with eyes like warm amber and a tail that seemed to have a life of its own. He was older, like her, with a dusting of grey around his muzzle, and a quiet dignity that belied his wagging enthusiasm.\n\nBuster, in his boundless affection, turned Eleanor\'s quiet world upside down. A chewed slipper here, a frantic chase after a squirrel there, and the constant, joyful thumping of his tail against the floorboards. Her morning walks